In [66]:
import pandas as pd 
import numpy as np
from scipy.sparse import load_npz,save_npz,lil_matrix,csr_matrix
from tqdm import tqdm 
import os

In [67]:
path = '.../data/baseline_plus_aux_data/matrices/cls/cls_T10_y.npz'
y = load_npz(path)

In [ ]:
y.shape

In [ ]:
path = '.../data/baseline_plus_aux_data/matrices/cls/cls_weights.csv'
weights = pd.read_csv(path)

In [ ]:
weights

In [71]:
# link this to an id in the summary eps 0.05 file

In [72]:
path = '.../data/baseline_plus_aux_data/results_tmp/classification/T8c.csv'
t8c = pd.read_csv(path)

In [ ]:
t8c.columns

In [74]:
path = '.../data/baseline_plus_aux_data/T0_image_pseudolabels_plus_baseline.csv'
t0 = pd.read_csv(path)

In [76]:
df_mgd = pd.merge(
            weights
            ,
            pd.merge(
                t0
                ,t8c
                ,how='inner'
                ,on='input_assay_id'
            )
            ,how='inner'
            ,left_on='task_id'
            ,right_on='cont_classification_task_id'
        )

In [80]:
path = './cp/summary_eps_0.05.csv'
df_cp = pd.read_csv(path)

In [81]:
df_mgd_cp = pd.merge(
    df_cp
    ,df_mgd
    ,how='inner'
    ,left_on='index'
    ,right_on='melloddy_col_idx'
)

In [87]:
path = '../analysis/cp/summary_eps_0.05_mgd_cp.csv'
df_mgd_cp.to_csv(path, index=False)

In [82]:
lst = [0
       ,0.2
       ,0.4
       ,0.5
       ,0.6
       ,0.7
       ,0.8
       ,0.9
       ,0.95
       ,0.99
      ]

In [ ]:
y_lil = lil_matrix(y)
all_cctis = set(df_mgd_cp['cont_classification_task_id'])
base_path = '/projects/home/wheyndri/research/wp1_imageauxdata/12_y3_imageaux_study/data/baseline_plus_aux_data/matrices/cls/confidence_selection/'
for i,l1 in tqdm(enumerate(lst)):
    for j,l2 in enumerate(lst): 
        file_core = 'ppv{}_npv{}'.format(l1,l2).replace('.','_')
        y_out = y_lil.copy()
        cctis_to_keep = set(df_mgd_cp.query('validity_0 > @l1').query('validity_1 > @l2')['cont_classification_task_id'])
        cctis_to_drop = all_cctis.difference(cctis_to_keep)
        for ccti in cctis_to_drop: 
            y_out[:,ccti] = 0
        save_npz(os.path.join(base_path,'cls_T10_y_'+file_core+'.npz')
                 ,csr_matrix(y_out))